In [1]:
import pandas as pd
from haystack_integrations.document_stores.qdrant import QdrantDocumentStore
import json
from haystack.dataclasses.document import Document


/Users/elisabeth/1_Lokale_Dateien/WS24-MyDailyImpact/deeplearning/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/elisabeth/1_Lokale_Dateien/WS24-MyDailyImpact/deeplearning/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
metadata_with_reviews = pd.read_csv("data/before_embedding_metadata_with_reviews.csv")
metadata = pd.read_csv("data/before_embedding_metadata.csv")
vegan_recipes = pd.read_csv("data/before_embedding_vegan_recipes.csv")
reviews_df = pd.read_csv("data/before_embedding_reviews_df.csv")

In [ ]:
vegan_recipes.iloc[0]

pd.set_option('display.max_columns', None)

# überprüfen ob Rezepte schon geladen wurden weil Nummer unter 100 ist:
list_documents = []
for i in range(0,100):
    recipe_id = vegan_recipes.iloc[i]["RecipeId"]
    recipe_text = vegan_recipes.iloc[i]["Name"]
    list_documents.append((recipe_id, recipe_text))
print(list_documents)

vegan_recipes[vegan_recipes['Name']== 'Lemon-Basil Dressing']
vegan_recipes[vegan_recipes['Name']== 'Peppy Salad Dressing']
vegan_recipes[vegan_recipes['Name']== 'Almond Paste']

,Unnamed: 0,RecipeId,Name,AuthorId,CookingTimeMinutes,PreparationTimeMinutes,TotalRecipeMinutes,Description,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,ReviewCount,Calories,FatContent,SaturatedFatContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeInstructions,RecipeTimeCategory,RatingReviewScore,merged_text
1,50,90,Almond Paste,1537,10080,15,10095,Description not provided. Please generate a de...,Dessert,"Fruit, Nuts, Lactose Free, Vegan, Low Choleste...","1, 3, 1, 2","fresh lemon juice, water, sugar",4.0,2.0,708.6,31.45,4.6,24.7,15.55,30.65,20.15,NaN,Force blanched almonds through fine blade of f...,Very Long,0.402375,Name: Almond Paste; Cooking Time Minutes: 1008...


In [20]:
print(vegan_recipes.index[vegan_recipes['Name']== 'Lemon-Basil Dressing'].tolist())
print(vegan_recipes.index[vegan_recipes['Name']== 'Healthy Pasta Salad'].tolist())
print(vegan_recipes.index[vegan_recipes['Name']== 'Sweet Sour Sauce Secrets'].tolist())
print(vegan_recipes.index[vegan_recipes['Name']== 'Tikka Spice Mix'].tolist())
print(vegan_recipes.index[vegan_recipes['Name']== 'Cappuccino Mix'].tolist())
print(vegan_recipes.index[vegan_recipes['Name']== 'Coffee Liqueur (Fake Kahlua)'].tolist())
print(vegan_recipes.index[vegan_recipes['Name']== 'Cranberry Sauce'].tolist()) #7)
print(vegan_recipes.index[vegan_recipes['Name']== 'Salsa Dip II'].tolist())
print(vegan_recipes.index[vegan_recipes['Name']== 'Vegetable Stock'].tolist())
print(vegan_recipes.index[vegan_recipes['Name']== 'Gazpacho 1 (adopted)'].tolist())
print(vegan_recipes.index[vegan_recipes['Name']== 'Sweetened Soy Sauce'].tolist()) #11
print(vegan_recipes.index[vegan_recipes['Name']== 'Easy Cranberry Sauce'].tolist())
print(vegan_recipes.index[vegan_recipes['Name']== 'Basic Cocktail Sauce'].tolist())
print(vegan_recipes.index[vegan_recipes['Name']== 'Stewed Tomatoes'].tolist())
print(vegan_recipes.index[vegan_recipes['Name']== 'Tofu Mayonnaise'].tolist())
print(vegan_recipes.index[vegan_recipes['Name']== 'Cranberry Relish II'].tolist())
print(vegan_recipes.index[vegan_recipes['Name']== 'Vfw Ladies Auxiliary Ceviche'].tolist())
print(vegan_recipes.index[vegan_recipes['Name']== 'Cabbage Soup'].tolist())
print(vegan_recipes.index[vegan_recipes['Name']== 'Strawberry Blender Smoothie'].tolist())
print(vegan_recipes.index[vegan_recipes['Name']== 'Low Calorie Lemonade'].tolist())
print(vegan_recipes.index[vegan_recipes['Name']== 'Cranberry Burgundy Sauce'].tolist())
print(vegan_recipes.index[vegan_recipes['Name']== 'Crispy Rice Fruit Balls'].tolist()) #22
print(vegan_recipes.index[vegan_recipes['Name']== 'Hot Norwegian Fruit Soup'].tolist())
print(vegan_recipes.index[vegan_recipes['Name']== 'Spaghetti Squash Salad'].tolist())
print(vegan_recipes.index[vegan_recipes['Name']== 'Hot and Sour Cabbage'].tolist())


[39]
[47]
[23]
[24]
[40]
[6]
[76]
[99]
[87]
[93]
[50]
[75]
[53]
[17]
[13]
[69]
[34]
[0]
[15]
[89]
[72]
[42]
[74]
[56]
[83]


In [15]:
vegan_recipes.iloc[19]['merged_text']

'Name: Peppy Salad Dressing; Cooking Time Minutes: 30; Preparation Time Minutes: 0; Total Recipe Minutes: 30; Description: Description not provided. Please generate a description.; Recipe Category: Salad Dressings; Keywords: "< 15 Mins"; Recipe Ingredient Quantities: 1, 1, 1/8, 1/8, 1/4, 1, 1; Recipe Ingredient Parts: sugar, dry mustard, pepper, water, vinegar, garlic; Calories: 9.1; Fat Content: 0.0; Saturated Fat Content: 0.0; Carbohydrate Content: 2.3; Fiber Content: 0.2; Sugar Content: 0.5; Protein Content: 0.0; Recipe Servings: 8.0; Recipe Instructions: Combine pectin sugar mustard and pepper. Stir in water vinegar and garlic.  Cover and chill for 1 hour. Chill to store two or three days. Variations: Herbed Salad Dressing: Prepare as above except add 1/8 teaspoon  dried basil crushed and 1/8 teaspoon paprika to the dry pectin mixture.  Continue as directed. Parmesan Salad Dressing: Prepare as above except add 1  tablespoon grated Parmesan cheese and 1/4 tsp. dried oregano crushed 

---


In [ ]:
with open("data/recipes_data.json", "r") as f:
    vector_recipes = json.load(f)

vector_recipes_dict = {elem['RecipeId']: elem['VectorData'] for elem in vector_recipes}

In [50]:
document_store = QdrantDocumentStore(
    "100.107.35.86",
    index="recipe_test", # anders nennen später
    embedding_dim=768,
    recreate_index=True, # nicht immer recreating
    return_embedding=True,
    wait_result_from_api=True,
)

In [51]:
list_documents = []
for i in range(0,100):
    recipe_id = vegan_recipes.iloc[i]["RecipeId"]
    recipe_text = vegan_recipes.iloc[i]["merged_text"]
    recipe_vector = vector_recipes_dict[recipe_id]

    list_documents.append(Document(content=recipe_text, embedding=recipe_vector))

document_store.write_documents(list_documents)

100%|██████████| 100/100 [00:00<00:00, 104.16it/s]


100

In [ ]:
# to see how many documents are in the document-store

# document_store.count_documents()

100

---

In [64]:
import json

vector_metadata = []
with open("data/metadata_with_reviews.json", "r") as f:
    for line in f:
        vector_metadata.append(json.loads(line.strip()))

In [71]:
vector_metadata[1]


{'RecipeId': 133,
 'AggregatedRating': 5.0,
 'ReviewCount': 4.0,
 'RatingReviewScore': 0.5047505938,
 'Name': [-0.0090075291,
  -0.0092835465,
  -0.0068938623,
  0.0310163963,
  0.044206717,
  0.0128210753,
  -0.0005102646,
  0.0554741509,
  -0.0263744917,
  0.0044761874,
  -0.0049482589,
  -0.0685538251,
  0.002505833,
  0.0426541773,
  -0.0206150459,
  0.0072264049,
  0.0094235397,
  0.0269177256,
  0.0380684176,
  0.0294704289,
  0.0141055596,
  -0.0573636147,
  -0.0258922123,
  0.0335314613,
  0.0280692189,
  -0.0064412669,
  -0.0271683399,
  0.0017660051,
  0.0185593288,
  -0.037817834,
  0.0011505959,
  0.0137768739,
  0.0485676222,
  -0.0085136945,
  0.0343909616,
  -0.0192918956,
  -0.0002362074,
  -0.0141610327,
  -0.0421805055,
  0.0168997168,
  -0.0484376617,
  -0.0171551966,
  0.0420581341,
  -0.0233450247,
  0.0297055977,
  -0.0119433934,
  -0.0492896127,
  -0.005775792,
  -0.0003776039,
  -0.0302163894,
  -0.0003224063,
  0.0413128382,
  -0.0143754166,
  -0.0275127086,
  